# Objective

To present an overview of the four important Agentic AI patterns - Reflection, Tool Use, Planning and Multi-Agent Collaboration.

# Setup

## Installation

In [3]:
! pip install -q openai\
                 langchain\
                 langchain-openai \
                 langchain-experimental

In [1]:
# ! pip install -q openai==1.55.3 \
#                  langchain==0.3.7 \
#                  langchain-openai==0.2.9 \
#                  langchain-experimental==0.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.

In [2]:
! pip install -q git+https://github.com/openai/swarm.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 31.0 MB/s eta 0:00:00


## Imports

In [4]:
import json

from openai import AzureOpenAI
from langchain import hub

from langchain.agents import create_react_agent, Tool, AgentExecutor
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_openai import AzureChatOpenAI
from langchain_experimental.utilities import PythonREPL

from swarm import Swarm, Agent

In [ ]:
with open('config-azure.json') as f:
    configs = f.read()

In [ ]:
creds = json.loads(configs)

In [6]:
from google.colab import userdata
azure_api_key = userdata.get('azure_api_key')

In [7]:
client = AzureOpenAI(azure_endpoint='https://azuse-mdpfnp0w-swedencentral.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2025-01-01-preview',
            api_key=azure_api_key,
            api_version='2025-01-01-preview')

In [8]:
model_name = 'gpt-4o-2'

In [ ]:
# client = AzureOpenAI(
#     api_key=creds['AZURE_OPENAI_KEY'],
#     azure_endpoint=creds['AZURE_OPENAI_ENDPOINT'],
#     api_version='2024-02-01'
# )

In [9]:
# llm = AzureChatOpenAI(
#     azure_endpoint=creds['AZURE_OPENAI_ENDPOINT'],
#     api_key=creds['AZURE_OPENAI_KEY'],
#     api_version="2024-02-01",
#     model="gpt-4o-mini",
#     temperature=0
# )

NameError: name 'creds' is not defined

In [10]:
llm = AzureChatOpenAI(
    azure_endpoint='https://azuse-mdpfnp0w-swedencentral.openai.azure.com/openai/deployments/gpt-4o-2/chat/completions?api-version=2025-01-01-preview',
    api_key=azure_api_key,
    api_version="2025-01-01-preview",
    model="gpt-4o-2",
    temperature=0
)

In [11]:
swarm_client = Swarm(client)

# Pattern 1: Reflection

Self-reflection - Ask the LLM to reflect on its own work to improve its answer.

Consider the following use case where the LLM is tasked to extract structured information from medical notes. However, instead of asking the LLM to directly provide the answer, we present the output from the generator LLM to a reflector LLM (in this case the same model). The feedback from the reflector is used by the generator to improve its answer.

In [12]:
main_model = critique_model = 'gpt-4o-2'

In [13]:
medical_note_data = """
Medical Notes:
---
Patient Name: Ms. Krishnaveni
Age: 45 years
Gender: Female

Chief Complaint:
Ms. Krishnaveni presented with complaints of persistent abdominal pain, bloating, and changes in bowel habits over the past two months.

History of Present Illness:
Ms. Krishnaveni reports experiencing intermittent abdominal pain, predominantly in the lower abdomen, accompanied by bloating and alternating episodes of diarrhea and constipation. She describes the pain as crampy in nature, relieved partially by defecation but worsening after meals. There is no association with specific food items. She denies any rectal bleeding, unintended weight loss, or fever.

Past Medical History:
Ms. Krishnaveni has a history of irritable bowel syndrome (IBS), diagnosed five years ago, managed with dietary modifications and occasional use of over-the-counter antispasmodics.

Medications:
She occasionally takes over-the-counter antispasmodics for symptomatic relief of abdominal discomfort related to IBS.

Family History:
There is no significant family history of gastrointestinal disorders or malignancies.

Social History:
Ms. Krishnaveni is a non-smoker and does not consume alcohol. She works as a teacher in a local school.
"""

The system message for the generator is below:

In [14]:
system_message = """
You are an expert assistant to a hospital administration team working on extracting important information from medical notes made by doctors.
Extract relevant information from the note presenteed by the user with the following schema.
- age: integer, age of the patient
- gender: string, can be one of male, female or other
- diagnosis: string, can be one of migraine, diabetes, arthritis and acne
- weight: integer, weight of the patient
- smoking: string, can be one of yes or no
Use information ONLY from the medical note to come up with the JSON output.

If you receive feedback from the user, use it to provide a revised version of your answer.
"""

Notice how the system message here explicitly acknowledges that feedback might be provided and should be used to improve the answer.

In [15]:
reflection_system_message = """
You are an expert assistant to a hospital administration team who is tasked to generate critique and recommendations for output from an LLM.
The input will contain an attempt by an LLM to extract relevant information in a JSON format of a medical note presented further below
The LLM was instructed that the JSON output needs to be extracted according to the following schema.
- age: integer, age of the patient
- gender: string, can be one of male, female or other
- diagnosis: string, can be one of migraine, diabetes, arthritis and acne
- weight: integer, weight of the patient
- smoking: string, can be one of yes or no

When you review the LLM attempt ensure that your critique is in accordance with the above schema.
While you are checking the input entered by the user, check if the input contains only the JSON and no additional information.
Provide explicit feedback if you notice additional information apart from the JSON.
Do not provide any suggestions for the output; restrict yourself to feedback.
---
{medical_note_data}
"""

Now let us run the first generation.

In [16]:
first_response = client.chat.completions.create(
    model=main_model,
    messages=[
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': medical_note_data}
    ],
    temperature=0.2
).choices[0].message.content

In [17]:
print(first_response)

```json
{
  "age": 45,
  "gender": "female",
  "diagnosis": "",
  "weight": null,
  "smoking": "no"
}
```


We will now present this output to the reflector that will present a critique according to our instructions above.

In [18]:
first_critique = client.chat.completions.create(
    model=critique_model,
    messages=[
        {'role': 'system', 'content': reflection_system_message},
        {'role': 'user', 'content': first_response}
    ],
    temperature=0
).choices[0].message.content

In [19]:
print(first_critique)

The JSON output provided has the following issues when evaluated against the schema:

1. **Diagnosis**: The "diagnosis" field is empty (`""`). According to the schema, it must be one of the specified values: "migraine," "diabetes," "arthritis," or "acne." An empty string is not valid.

2. **Weight**: The "weight" field is `null`. According to the schema, this field must be an integer. A `null` value does not comply with the schema requirements.

The rest of the fields ("age," "gender," and "smoking") are correctly formatted and adhere to the schema.

Additionally, the input contains only the JSON and no extraneous information, which is correct.


As can be seen from the above output, the reflector identified several issues with the output. We can now present this critique as feedback to the original generator so it can amend its response.

In [20]:
second_response = client.chat.completions.create(
    model=main_model,
    messages=[
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': medical_note_data},
        {'role': 'assistant', 'content': first_response},
        {'role': 'user', 'content': first_critique}
    ],
    temperature=0.2
).choices[0].message.content

In [21]:
print(second_response)

Thank you for the feedback. Based on the schema and the issues you pointed out, I will revise the JSON output. However, the medical note provided does not contain information about the patient's weight or a diagnosis that matches the specified options ("migraine," "diabetes," "arthritis," or "acne"). Since the schema requires these fields to be filled, I will provide placeholders where appropriate.

Here is the revised JSON output:

```json
{
  "age": 45,
  "gender": "female",
  "diagnosis": null,
  "weight": null,
  "smoking": "no"
}
```

Explanation of changes:
1. **Diagnosis**: Set to `null` because the medical note does not mention any of the specified diagnoses.
2. **Weight**: Set to `null` because the medical note does not provide the patient's weight.

If you have additional information or further feedback, please let me know!


As we can see from the output above, the critique can be used to improve the response over a series of reflective interventions.

# Pattern 2: Tool Use

Let us see how tool use can augement LLM capabilities with a simple example. First, we beign by defining a series of Python functions that we then wrap as LangChain tools using the `@tool` decorator.

In [23]:
@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """

    return a * b

@tool
def subtract(a: int, b: int) -> int:
    """Subtract b from a

    Args:
        a: first int
        b: second int
    """

    return a - b

Note that the doc strings of the function describing what the functions do is a critical input parsed by the tool to understand which function needs to be called when a user input is received.

We then collect these tools into a dictionary with function names as the keys like so:

In [24]:
available_tools = {
    'add': add,
    'multiply': multiply,
    'subtract': subtract
}

We can now create a basic agent by binding the LLM with these three tools.

In [25]:
agent = llm.bind_tools(list(available_tools.values()))

Now our agent is capable of answering questions that can be resolved as evaluations of the three functions available to it as tools. Consider the following user query.

In [26]:
query = "Which is bigger: 3 * 12 or 11 + 49? what is the difference between the two"

messages = [HumanMessage(query)]

In [27]:
agent.invoke(messages).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_9BskPg6i9m0nYgvqQXaFnMWh',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_igrZTEY8gTJOGaDIO5AHIcuH',
  'type': 'tool_call'}]

Notice how the LLM behavior changed. Instead of answering the question correctly, it has composed a tool call output that is an intermediate step to answer the question from the user.

Specifically, it has correctly recognized that it has to call the functions `multiply` and `add` with the correct arguments in order to answer the user question.

Note that this is still a partial execution of a tool-calling agent. We will see an end-to-end execution of the tool-calling agent in a future session.

# Pattern 3: Planning

Planning agents utilize a specified algorithm to plan/structure their efforts to achieve a business objective. Let us see an example of a Reasoning and Action (ReAct) agent. We will take a much deeper look into ReAct agents in upcoming sessions.

In [28]:
react_prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [30]:
print(react_prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


As the above prompt template indicates, the LLM is asked to 'think' through before answering in a Thought/Action/Observation sequence till a final answer is reached.

With this prompt, let us now create a simple Python agent that will always use the Python interpretor to answer user queries.

In [32]:
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell used to execute python commands. Input should be a valid python command.",
    func=python_repl.run,
)

As we have seen in previous examples, the `repl_tool` is basically a wrapper around the `python_repl` function. We can now create the ReAct agent by binding this `repl_tool` to the LLM like so:

In [33]:
react_agent = create_react_agent(
    llm=llm,
    tools=[repl_tool],
    prompt=react_prompt
)

In `LangChain` agent execution is handled by executors that track the tool calls and execute them in dedicated threads.

In [34]:
react_agent_executor = AgentExecutor(
    agent=react_agent,
    tools=[repl_tool],
    verbose=True
)

Let us now test our Python tool-calling agent with a non-trivial math problem.

In [35]:
user_input = "If USD 450 amounts to USD 630 in 6 years, what will it amount to in 2 years at the same interest rate?"

In [36]:
react_agent_executor.invoke(
    {
        'input': user_input
    }
)



> Entering new AgentExecutor chain...


This is a problem involving simple interest. To solve it, we need to determine the interest rate and then calculate the amount for 2 years using the same rate.

The formula for simple interest is:

\[
A = P(1 + rt)
\]

Where:
- \(A\) is the total amount (principal + interest),
- \(P\) is the principal,
- \(r\) is the rate of interest per year,
- \(t\) is the time in years.

From the given data:
- \(P = 450\),
- \(A = 630\),
- \(t = 6\).

We can first calculate the interest rate \(r\), and then use it to find the amount for \(t = 2\).

Action: python_repl
Action Input: "P = 450; A = 630; t = 6; r = (A / P - 1) / t; A_2 = P * (1 + r * 2); A_2"It seems I forgot to wait for the result of the calculation. Let me re-run the computation to determine the amount for 2 years.

Action: python_repl
Action Input: "P = 450; A = 630; t = 6; r = (A / P - 1) / t; A_2 = P * (1 + r * 2); A_2"It seems I made an error in my process. Let me carefully re-run the computation to determine the amount for 2 year

{'input': 'If USD 450 amounts to USD 630 in 6 years, what will it amount to in 2 years at the same interest rate?',
 'output': 'The amount after 2 years will be approximately **USD 510.03**.'}

# Pattern 4: Multi-agent Collaboration

In one pattern of multi-agent collaboration called the `Triage` mode, a focal agent is tasked with handing-off tasks to appropriate agents. As an example, consider the following scenario where there are two agents - A & B. Agent A is the main agent that has two tools - a function to greet customers and to transfer control to Agent B. Agent B can only speak in Hindi, but can only be reached when Agent A hands-off control.

Whether the control needs to reach Agent B or not is decided by Agent A depending on the user query.

We will look at many more patterns of multi-agent collaborations in an upcoming session.

In [37]:
def transfer_to_agent_b():
    return agent_b

In [38]:
def transfer_to_agent_a():
    return agent_a

In [39]:
def greet_customer():
    return "Hello, how can I help you?"

In [40]:
agent_a = Agent(
    name="Agent A",
    instructions="You are a helpful agent.",
    model='gpt-4o-mini',
    functions=[transfer_to_agent_b, greet_customer]
)

agent_b = Agent(
    name="Agent B",
    instructions="Only speak in Hindi.",
    model='gpt-4o-mini',
    functions=[transfer_to_agent_a]
)

In [48]:
response = swarm_client.run(
    agent=agent_a,
    messages=[{"role": "user", "content": "I need to speak to Agent B"}],
)

In [49]:
print(response.messages[-1]["content"])

ऐसा लगता है कि एजेंट B से संपर्क करना संभव नहीं है। यदि आप चाहें, तो मैं आपकी सहायता कर सकता हूं।


In [50]:
len(response.messages)

3

In [51]:
for message in response.messages:
    print(message)
    print('----')

{'content': None, 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_78REKlFPKJCBO0lHmX66skWH', 'function': {'arguments': '{}', 'name': 'transfer_to_agent_b'}, 'type': 'function'}], 'sender': 'Agent A'}
----
{'role': 'tool', 'tool_call_id': 'call_78REKlFPKJCBO0lHmX66skWH', 'tool_name': 'transfer_to_agent_b', 'content': '{"assistant": "Agent B"}'}
----
{'content': 'ऐसा लगता है कि एजेंट B से संपर्क करना संभव नहीं है। यदि आप चाहें, तो मैं आपकी सहायता कर सकता हूं।', 'refusal': None, 'role': 'assistant', 'annotations': [], 'audio': None, 'function_call': None, 'tool_calls': None, 'sender': 'Agent B'}
----
